In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [2]:
import os
os.chdir("/content/gdrive/My Drive/proj/config")
!ls

cal_ssim.py  eval.ipynb  __pycache__  show.py	      train.py
clean.sh     eval.py	 settings.py  tensorboard.sh
dataset.py   model.py	 show.ipynb   train.ipynb


In [3]:
import torch
a = torch.Tensor([1]).cuda()
print(a)
torch.cuda.is_available()

tensor([1.], device='cuda:0')


True

In [4]:
!pip3 install torch torchvision
!pip3 install tensorboardX

     |████████████████████████████████| 204kB 20.4MB/s 


In [0]:
import os
import sys
import cv2
import argparse
import numpy as np
import itertools
from tqdm.notebook import tqdm
import torch
from torch import nn
from torch.nn import DataParallel
from torch.optim import Adam
from torch.autograd import Variable 
from torch.utils.data import DataLoader

import settings
from dataset import ShowDataset
from model import RESCAN 

logger = settings.logger
torch.cuda.manual_seed_all(66)
torch.manual_seed(66)
torch.cuda.set_device(settings.device_id)

In [0]:
def ensure_dir(dir_path):
    if not os.path.isdir(dir_path):
        os.makedirs(dir_path)

In [7]:
print(settings.model_dir)

../models/Rain100H


In [0]:
class Session:
    def __init__(self):
        self.show_dir = settings.show_dir
        self.model_dir = settings.model_dir
        ensure_dir(settings.show_dir)
        ensure_dir(settings.model_dir)
        logger.info('set show dir as %s' % settings.show_dir)
        logger.info('set model dir as %s' % settings.model_dir)

        self.net = RESCAN().cuda()
        self.dataset = None 
        self.dataloader = None 

    def get_dataloader(self, dataset_name):
        self.dataset = ShowDataset(dataset_name)
        print('dataset',self.dataset)
        self.dataloader = DataLoader(self.dataset, batch_size=1, 
                            shuffle=False, num_workers=1)
        return self.dataloader

    def load_checkpoints(self, name):
        ckp_path = os.path.join(self.model_dir, name)
        try:
            obj = torch.load(ckp_path)
            logger.info('Load checkpoint %s' % ckp_path)
        except FileNotFoundError:
            logger.info('No checkpoint %s!!' % ckp_path)
            return
        self.net.load_state_dict(obj['net'])

    def inf_batch(self, name, batch):
        O = batch['O'].cuda()
        O = Variable(O, requires_grad=False)

        with torch.no_grad():
            O_Rs = self.net(O)
            O_Rs = [O - O_R for O_R in O_Rs]
        
        return O_Rs

    def save_image(self, No, imgs):
        for i, img in enumerate(imgs):
            img = (img.cpu().data * 255).numpy()
            img = np.clip(img, 0, 255)[0]
            img = np.transpose(img, (1, 2, 0))
            h, w, c = img.shape

            img_file = os.path.join(self.show_dir, '%s_%d.png' % (No, i))
            cv2.imwrite(img_file, img)

In [0]:
def run_show(ckp_name='latest'):
    sess = Session()
    sess.load_checkpoints(ckp_name)
    sess.net.eval()

    dt = sess.get_dataloader('test')


    for i, batch in enumerate(dt):
        print(i)
        logger.info(i)
        imgs = sess.inf_batch('test', batch)
        No = sess.dataset.get_name(batch['idx'][0])
        sess.save_image(No, imgs)

In [10]:
run_show()

2020-05-09 23:14:20,244 - INFO - set show dir as ../showdir/Rain100H
2020-05-09 23:14:20,245 - INFO - set model dir as ../models/Rain100H
2020-05-09 23:14:21,207 - INFO - Load checkpoint ../models/Rain100H/latest


dataset <dataset.ShowDataset object at 0x7f48f809cdd8>


2020-05-09 23:14:22,765 - INFO - 0


0


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
2020-05-09 23:14:25,839 - INFO - 1


...

7


2020-05-09 23:14:46,647 - INFO - 8


8


2020-05-09 23:14:49,137 - INFO - 9


9
